
***Assignment 3***
---

***Submitted by Rahul Raju Pattar***

*Write a function for calculating the value of a European option on Nifty using Monte Carlo Simulation for an expiry after a month. Compare it with the current market price and tell your decision, if it shall be bought or sold.*



**Discussion:**

The main assumptions in calculating the value of a European Option on Nifty using Monte Carlo Simulation for an expiry after a month are:

**1.** Asset returns are normally distributed.\
**2.** Asset returns are assumed to be serially independent in that no prior return should influence the current return.

In the following, the main procedure for the calculating parametric VaR of the portfolio consisting of the given stocks is outlined:

**Step 1.** Fetch the real time data for: (1) Nifty price (2) Volatility (3) Risk-free rate\
**Step 2.** Simulate the paths for European option prices using Monte-Carlo method and find the option prices.\
**Step 3.** Compare the simulated prices with the market prices to decide whether to buy or sell.


In [8]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import scipy.stats as stats
from datetime import datetime, timedelta

In [2]:
#Market prices for European call and put options

# Market Price for call option
market_price_call = 284.00

# Market Price for put option
market_price_put = 589.95

***STEP 1***
------


In [3]:
#Function to fetch real time data for nifty price, volatility and risk free rate:

def fetch_real_time_data():
    
    # Fetch current Nifty price
    nifty = yf.Ticker("^NSEI")
    nifty_price = nifty.history(period="1d")['Close'].iloc[-1].round(4)
    
    # Fetch historical data to estimate volatility
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365)
    historical_data = nifty.history(start=start_date, end=end_date)
    
    #Compute historical daily returns
    historical_returns = np.log(historical_data['Close']).diff().dropna()
    
    #Compute annualized volatility from the historical daily returns
    volatility = (historical_returns.std() * np.sqrt(252)).round(4)  # Annualized volatility

    # Risk-free rate taken from RBI website's 10-Year G-Sec Par Yield (FBIL)
    # Source: 'https://www.rbi.org.in/Scripts/BS_NSDPDisplay.aspx?param=4'
    risk_free_rate = 0.0701

    return nifty_price, volatility, risk_free_rate

In [4]:
'''
  Parameters:
  
  S0             : initial stock price
  sigma          : annulaized volatility
  risk_free_rate : risk free rate
  K              : Strike Price
  T              : Time to expiry (1 month)
  N              : Number of simulation paths for Monte Carlo simulation
  
'''
S0, sigma, risk_free_rate = list(fetch_real_time_data())       # Fetch real-time data
K = 24600                                                
T = 1/12                                                 
N = 10_000                                                

***STEP 2***
------

In [5]:
# Function to simulate European option prices using Monte-Carlo method

def monte_carlo_european_option(S0, K, T, r, sigma, N = 10_000):
    
    # Generate N random paths for the asset price
    Z = np.random.standard_normal(N)
    ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)
    
    # Calculate the payoff at expiry
    call_payoff = np.maximum(ST - K, 0)
    put_payoff = np.maximum(K-ST,0)
    
    # Discount the average payoff back to present value
    call_price = (np.exp(-r * T) * np.mean(call_payoff)).round(4)
    put_price = (np.exp(-r * T) * np.mean(put_payoff)).round(4)
    
    return call_price, put_price

In [6]:
# Calculate the option price using Monte Carlo simulation
simulated_call_price, simulated_put_price = monte_carlo_european_option(S0, K, T, risk_free_rate, sigma, N)

print(f"\nSimulated Call Option Price: {simulated_call_price}")
print(f"Market Price for Call Option: {market_price_call}\n")

print(f"Simulated Put Option Price: {simulated_put_price}")
print(f"Market Price for Put Option: {market_price_put}\n")


Simulated Call Option Price: 175.8149
Market Price for Call Option: 284.0

Simulated Put Option Price: 633.7144
Market Price for Put Option: 589.95



***STEP 3***
-----------

In [7]:
# Decision making for CALL OPTION
if simulated_call_price > market_price_call:
    decision_call = "Buy the CALL option as it is undervalued."
else:
    decision_call = "Sell the CALL option as it is overvalued."
    
# Decision making for PUT OPTION
if simulated_put_price > market_price_put:
    decision_put = "Buy the PUT option as it is undervalued."
else:
    decision_put = "Sell the PUT option as it is overvalued."

print(f"Decision for CALL Option : {decision_call}")
print(f"Decision for PUT Option : {decision_put}")

Decision for CALL Option : Sell the CALL option as it is overvalued.
Decision for PUT Option : Buy the PUT option as it is undervalued.
